In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft


  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-8h6vwqv4/audiocraft_ccb4c730f2c845db96c690f01f697b0e
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-8h6vwqv4/audiocraft_ccb4c730f2c845db96c690f01f697b0e
  Resolved https://github.com/facebookresearch/audiocraft to commit f5931855b8e662462d0af8256d9c084ca04d6a94
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 47.3 MB/s eta 0:00:00
  Preparing m

In [ ]:
!pip install audiocraft

In [ ]:
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

In [ ]:
model = musicgen.MusicGen.get_pretrained('small', device='cuda')
# model.set_generation_params(duration=8)

/usr/local/lib/python3.10/dist-packages/audiocraft/models/musicgen.py:83: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-small
  warnings.warn(


state_dict.bin:   0%|          | 0.00/841M [00:00<?, ?B/s]

compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [ ]:
res = model.generate([
    'crazy EDM, heavy bang',
    'classic reggae track with an electronic guitar solo',
    'lofi slow bpm electro chill with organic samples',
    'rock with saturated guitars, a heavy bass line and crazy drum break and fills.',
    'earthy tones, environmentally conscious, ukulele-infused, harmonic, breezy, easygoing, organic instrumentation, gentle grooves',
],
    progress=True)
display_audio(res, 32000)

# SFT

In [ ]:
import torch
import os
from torch.utils.data import DataLoader, Dataset
from audiocraft.models import musicgen
from audiocraft.losses import STFTLoss

class MusicBenchDataset(Dataset):
    def __init__(self, data_path):
        self.data_path = data_path
        self.samples = []
        self._load_data()

    def _load_data(self):
        for root, _, files in os.walk(self.data_path):
            for file in files:
                if file.endswith(".wav") or file.endswith(".mp3"):
                    self.samples.append(os.path.join(root, file))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        audio_path = self.samples[idx]
        audio, text = self._load_audio(audio_path)
        return text, audio

In [ ]:
model = musicgen.MusicGen.get_pretrained('small', device='cuda')

learning_rate = 1e-4
batch_size = 8
epochs = 5

model.set_train_mode()  # Перевод модели в режим обучения
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = STFTLoss()

In [ ]:
train_dataset = MusicBenchDataset(data_path="musicbench")
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for i, (text, audio) in enumerate(train_dataloader):
        optimizer.zero_grad()

        inputs = model.tokenizer(text)
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
        audio = audio.to('cuda')

        outputs = model(inputs)
        loss = criterion(outputs, audio)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Batch {i}/{len(train_dataloader)}, Loss: {loss.item():.4f}")

    print(f"Epoch {epoch + 1} completed. Average Loss: {running_loss / len(train_dataloader):.4f}")

output_path = "/model/finetuned_model.pth"
torch.save(model.state_dict(), output_path)
print(f"Model saved to {output_path}")